In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
import time
import datetime

def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))


def key_replace(item):
    new_key, current_path = item[1][1], item[1]
    return (new_key,current_path)

def join_build(item):
    join_key, holder_value = item[0],item[1][0]
    return (join_key, holder_value)

def new_value(item):
    old_key, current_value = item[0],item[1][1]
    return (old_key,current_value)

def build_path(item):
    next_key, path, value = item[1][1], item[1][0], item[1][1]
    path = list(path)
    if not value in path: #added this line to check for list containment of new connection
        path.append(value)
        return (next_key,(path))
    else:
        return (-1,(path))

def final_RDD(item):
    return(item[1])

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

x = 12
d = 12
distances = sc.parallelize([(x, d)]).partitionBy(n)

candidates = distances.join(forward_edges, n).cache()
new_candidate = candidates.map(key_replace).cache()
connection = new_candidate.map(join_build).cache()
new_pair = connection.join(forward_edges,n).map(new_value).cache()
merged_path_value = new_candidate.join(new_pair,n).cache()
new_full_key_and_path = merged_path_value.map(build_path).cache()

a = 1

while True:
#    start_ts = time.time()
#    start = datetime.datetime.fromtimestamp(start_ts).strftime('%Y-%m-%d %H:%M:%S')
#    print("Iteration number:",a," starting at",start,"\n")
    connection = new_full_key_and_path.map(join_build).cache()
#    print("new connection:", connection.collect(),"\n")
    new_pair = connection.join(forward_edges,n).map(new_value).cache()
#    print("new new_pair:", new_pair.collect(),"\n")
    new_full_key_and_path_2 = new_full_key_and_path.join(new_pair,n).cache()
#    print("new temp path:", new_full_key_and_path_2.collect(),"\n")
    new_full_key_and_path = new_full_key_and_path_2.map(build_path).cache()
#    print("next temp path:",new_full_key_and_path.collect(),"\n")
#    stop_ts = time.time()
#    stop = datetime.datetime.fromtimestamp(stop_ts).strftime('%Y-%m-%d %H:%M:%S')
#    print("Iteration finished at", stop,"\n")
    a += 1
    if not new_full_key_and_path.lookup(34):
        continue
    else:
        break
        
final_RDD = new_full_key_and_path.filter(lambda i:i[0]==34).map(final_RDD)

answer = final_RDD.take(9)

answer_list = []
for i in answer:
    if not i==34:
        answer_list.append(i)

no_space = str(answer_list)
no_space = no_space.replace(",34","")
no_space = no_space.replace(" ","")
no_space = no_space.replace('[','')
no_space = no_space.replace(']','')
print(no_space)

12,422,53,52,107,20,23,274,34
